In [480]:
import pickle as pkl
import dill as pickle
from nltk.translate.phrase_based import phrase_extraction,extract
from nltk.translate import AlignedSent
from nltk.translate import PhraseTable
from collections import defaultdict
import json
import os
import re
import string

from cltk.tokenize.word import WordTokenizer
from cltk.stem.lemma import LemmaReplacer
from nltk.translate import AlignedSent, Alignment, IBMModel1, IBMModel2
from nltk.tokenize import wordpunct_tokenize
from nltk.translate.ibm_model import AlignmentInfo

In [473]:
from numpy import log

In [9]:
lat_tokenizer = WordTokenizer('latin').tokenize
eng_tokenizer = wordpunct_tokenize
lem = LemmaReplacer('latin')

In [413]:
with open('../pkls/lat_ibm_model_2_bitext.pk', 'rb') as fin:
    lbitext = pkl.load(fin)

with open('../pkls/eng_ibm_model_2_bitext.pk', 'rb') as fin:
    ebitext = pkl.load(fin)

In [414]:
for b in range(len(ebitext)):
    if ebitext[b].mots[-1] != 'NULL':
        ebitext[b].mots.append('NULL')
    if ebitext[b].words[-1] != 'NULL':
        ebitext[b].words.append('NULL')
    lens = [len(ebitext[b].words),len(ebitext[b].mots)]

    alin = [list(tup) for tup in ebitext[b].alignment]

    for i,a in enumerate(alin):
        if None in a:
            alin[i][alin[i].index(None)] = lens[a.index(None)]-1

    alin = [tuple(tup) for tup in alin]

    ebitext[b].alignment = Alignment(alin)

In [415]:
for b in range(len(lbitext)):
    if lbitext[b].mots[-1] != 'NULL':
        lbitext[b].mots.append('NULL')
    if lbitext[b].words[-1] != 'NULL':
        lbitext[b].words.append('NULL')
    lens = [len(lbitext[b].words),len(lbitext[b].mots)]

    alin = [list(tup) for tup in lbitext[b].alignment]

    for i,a in enumerate(alin):
        if None in a:
            alin[i][alin[i].index(None)] = lens[a.index(None)]-1

    alin = [tuple(tup) for tup in alin]

    lbitext[b].alignment = Alignment(alin)

In [416]:
skip = [7829, 9778,11352]

ephrases = defaultdict(lambda: [0,defaultdict(int)])

for b in range(len(ebitext)): 
    if b in skip:
        continue
    pe = phrase_extraction(' '.join(ebitext[b].words), ' '.join(ebitext[b].mots), ebitext[b].alignment,max_phrase_length=4)
    pe = list(pe)
    for phrase in pe:
        english_tokens = eng_tokenizer(phrase[2])
        lat_tokens = lat_tokenizer(phrase[3])
        
        if len(english_tokens) < 5 and len(lat_tokens) < 5:
            ephrases[tuple(english_tokens)][1][tuple(lat_tokens)] += 1
            ephrases[tuple(english_tokens)][0] += 1

In [417]:
#skip = [7829, 9778,11352]

lphrases = defaultdict(lambda: [0,defaultdict(int)])

for b in range(len(lbitext)): 
    if b in skip:
        continue
    pe = phrase_extraction(' '.join(lbitext[b].words), ' '.join(lbitext[b].mots), lbitext[b].alignment,max_phrase_length=4)    
    pe = list(pe)
    for phrase in pe:
        english_tokens = eng_tokenizer(phrase[3])
        lat_tokens = lat_tokenizer(phrase[2])
        
        if len(english_tokens) < 5 and len(lat_tokens) < 5:
            lphrases[tuple(lat_tokens)][1][tuple(english_tokens)] += 1
            lphrases[tuple(lat_tokens)][0] += 1

In [418]:
phrases = defaultdict(lambda: [0,defaultdict(int)])
untranslated = dict()

for phrase in list(ephrases.keys()):
    added = False
    for trg_phrase in ephrases[phrase][1]:
        if lphrases[trg_phrase][0] != 0:
            for sub_phrase in lphrases[trg_phrase][1]:
                if sub_phrase == phrase:
                    phrases[phrase][1][trg_phrase] += ephrases[phrase][1][trg_phrase]
                    phrases[phrase][0] += ephrases[phrase][1][trg_phrase]
                    added = True
    if not added:
        untranslated[phrase] = ephrases[phrase]

In [447]:
phrases[('lives',)]#[('NULL'),]

[2, defaultdict(int, {('vivit',): 2})]

In [453]:
for k in untranslated:  # while there are still untranslated phrases
    maximum = 0
    for possible_translation in untranslated[k][1]:
        if untranslated[k][1][possible_translation] > maximum:
            maximum = untranslated[k][1][possible_translation]
            translation = possible_translation
    phrases[k][0] = maximum
    phrases[k][1][translation] = maximum

In [451]:
untranslated[k]

[3,
 defaultdict(int,
             {('clementia', 'caesaris'): 1,
              ('clementia', 'caesaris', 'reditum'): 1,
              ('clementia', 'caesaris', 'reditum', 'patere'): 1})]

In [468]:
phrases[('the',)]

[2259,
 defaultdict(int,
             {('-', 'ne'): 76,
              ('-', 'que'): 152,
              ('-', 'que', 'NULL'): 2,
              ('-', 'que', 'eo'): 1,
              ('-', 'que', 'id'): 1,
              ('-', 'que', 'in'): 3,
              ('a',): 35,
              ('ab',): 18,
              ('ac',): 34,
              ('ad',): 76,
              ('atque',): 35,
              ('autem',): 69,
              ('cum',): 37,
              ('ea',): 15,
              ('eius',): 18,
              ('eo',): 61,
              ('esse',): 7,
              ('est',): 233,
              ('est', 'NULL'): 58,
              ('et',): 215,
              ('et', 'in'): 1,
              ('ex',): 128,
              ('ex', 'eo'): 2,
              ('ex', 'quo'): 5,
              ('his',): 24,
              ('id',): 16,
              ('id', '-', 'que'): 5,
              ('in',): 264,
              ('inter',): 44,
              ('modo',): 18,
              ('non',): 18,
              ('per',): 42,
      

In [474]:
phrase_table = PhraseTable()

In [475]:
for k in phrases:
    for t in phrases[k][1]:
        phrase_table.add(k,t,log(phrases[k][1][t]/phrases[k][0]))

In [476]:
phrase_table.translations_for(('house',))

[PhraseTableEntry(trg_phrase=('domum',), log_prob=-0.20763936477824449),
 PhraseTableEntry(trg_phrase=('domus',), log_prob=-1.6739764335716716)]

In [481]:
with open('../pkls/latin_lang_model.pk', 'rb') as fin:
    lang_model = pickle.load(fin)

KeyboardInterrupt: 

In [478]:
from nltk.translate.stack_decoder import StackDecoder

In [ ]:
def language_prob(phrase):
    phrase = ['**','**'] + phrase + ['']

In [ ]:
language_model = type('',(object,),{'probability_change': lambda self, context, phrase: language_prob(phrase), 'probability': lambda self, phrase: language_prob(phrase)})()


In [ ]:
StackDecoder()